In [1]:
import timm
import numpy as np
import pandas as pd
import time
import torch
from tqdm.notebook import tqdm 

#import torch.utils.benchmark as benchmark
from torch.utils.data import Dataset, DataLoader

In [2]:
WARM_UP = 10
BATCH_SIZE = 4
NUM_TEST = 100

In [3]:
!nvidia-smi

Wed Dec 22 14:44:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.00    Driver Version: 470.82.00    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:09:00.0 Off |                  Off |
| 30%   47C    P8    17W / 300W |     33MiB / 48682MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
df_models = pd.read_csv("results-imagenet-real.csv")
# use models with img size 224
modellist = df_models[df_models["img_size"]==224]["model"]
df_models.head()

,model,top1,top1_err,top5,top5_err,param_count,img_size,cropt_pct,interpolation,top1_diff,top5_diff,rank_diff
0,beit_large_patch16_512,90.695,9.305,98.770,1.230,305.67,512,1.000,bicubic,2.111,0.110,0
1,beit_large_patch16_384,90.601,9.399,98.777,1.223,305.00,384,1.000,bicubic,2.219,0.169,0
2,tf_efficientnet_l2_ns,90.572,9.428,98.779,1.221,480.31,800,0.960,bicubic,2.226,0.125,0
3,tf_efficientnet_l2_ns_475,90.527,9.473,98.706,1.294,480.31,475,0.936,bicubic,2.289,0.156,0
4,beit_base_patch16_384,90.388,9.612,98.730,1.270,86.74,384,1.000,bicubic,3.580,0.590,4


In [5]:
class RandomDataset(Dataset):
    def __init__(self,  length, imsize):
        self.len = length
        self.data = torch.randn( 3, imsize, imsize, length)

    def __getitem__(self, index):
        return self.data[:,:,:,index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(BATCH_SIZE*(WARM_UP + NUM_TEST), 224),
                         batch_size=BATCH_SIZE, shuffle=False,num_workers=8)

In [6]:
# https://github.com/ryujaehun/pytorch-gpu-benchmark/blob/master/benchmark_models.py
def inference(modelname, benchmark, half=False):
    with torch.no_grad():
        model = timm.create_model(modelname,)
        model=model.to('cuda')
        model.eval()
        precision = "float"
        durations = []
        print(f'Benchmarking Inference {modelname} ')
        for step,img in enumerate(rand_loader):
            img=getattr(img,precision)()
            torch.cuda.synchronize()
            start = time.time()
            model(img.to('cuda'))
            torch.cuda.synchronize()
            end = time.time()
            if step >= WARM_UP:
                durations.append((end - start)*1000)
        print(f'{modelname} model average inference time : {sum(durations)/len(durations)}ms')
        
        if half:
            durations_half = []
            print(f'Benchmarking Inference half precision type {modelname} ')
            model.half()
            precision = "half"
            for step,img in enumerate(rand_loader):
                img=getattr(img,precision)()
                torch.cuda.synchronize()
                start = time.time()
                model(img.to('cuda'))
                torch.cuda.synchronize()
                end = time.time()
                if step >= WARM_UP:
                    durations_half.append((end - start)*1000)
            print(f'{modelname} half model average inference time : {sum(durations_half)/len(durations_half)}ms')
            
        if half:
            benchmark[modelname] = {"fp32": np.mean(durations), "fp16": np.mean(durations_half), "top1": df_models[df_models["model"]==modelname]["top1"]}
        else:
            benchmark[modelname] = {"fp32": np.mean(durations), "top1": float(df_models[df_models["model"]==modelname]["top1"])}
    return benchmark

In [7]:
benchmark = {}

# inference float precision
for i,modelname in tqdm(enumerate((modellist))):
    try:
        benchmark = inference(modelname, benchmark)
    except:
        print("pass {}".format(modelname))
benchmark

Benchmarking Inference beit_large_patch16_224 
beit_large_patch16_224 model average inference time : 21.222758293151855ms
Benchmarking Inference swin_large_patch4_window7_224 
swin_large_patch4_window7_224 model average inference time : 17.0994234085083ms
Benchmarking Inference xcit_large_24_p8_224_dist 
xcit_large_24_p8_224_dist model average inference time : 66.53245449066162ms
Benchmarking Inference beit_base_patch16_224 
beit_base_patch16_224 model average inference time : 7.464489936828613ms
Benchmarking Inference vit_large_patch16_224 
vit_large_patch16_224 model average inference time : 20.04459857940674ms
Benchmarking Inference xcit_medium_24_p8_224_dist 
xcit_medium_24_p8_224_dist model average inference time : 40.78274726867676ms
Benchmarking Inference xcit_small_24_p8_224_dist 
xcit_small_24_p8_224_dist model average inference time : 29.373767375946045ms
Benchmarking Inference swin_base_patch4_window7_224 
swin_base_patch4_window7_224 model average inference time : 12.806141

{'beit_large_patch16_224': {'fp32': 21.222758293151855, 'top1': 90.157},
 'swin_large_patch4_window7_224': {'fp32': 17.0994234085083, 'top1': 89.796},
 'xcit_large_24_p8_224_dist': {'fp32': 66.53245449066162, 'top1': 89.519},
 'beit_base_patch16_224': {'fp32': 7.464489936828613, 'top1': 89.438},
 'vit_large_patch16_224': {'fp32': 20.04459857940674, 'top1': 89.308},
 'xcit_medium_24_p8_224_dist': {'fp32': 40.78274726867676, 'top1': 89.286},
 'xcit_small_24_p8_224_dist': {'fp32': 29.373767375946045, 'top1': 89.207},
 'swin_base_patch4_window7_224': {'fp32': 12.806141376495361, 'top1': 89.19},
 'ig_resnext101_32x32d': {'fp32': 132.56885290145874, 'top1': 89.109},
 'ig_resnext101_32x48d': {'fp32': 210.10546445846558, 'top1': 89.107},
 'xcit_large_24_p16_224_dist': {'fp32': 19.736363887786865, 'top1': 89.045},
 'resmlp_big_24_224_in22ft1k': {'fp32': 33.9638614654541, 'top1': 89.019},
 'xcit_small_12_p8_224_dist': {'fp32': 16.07722043991089, 'top1': 89.007},
 'vit_base_patch16_224': {'fp32':

In [8]:
df_results = pd.DataFrame(benchmark).T
df_results

,fp32,top1
beit_large_patch16_224,21.222758,90.157
swin_large_patch4_window7_224,17.099423,89.796
xcit_large_24_p8_224_dist,66.532454,89.519
beit_base_patch16_224,7.464490,89.438
vit_large_patch16_224,20.044599,89.308
...,...,...
tf_mobilenetv3_small_100,5.031617,74.736
dla46x_c,4.911094,73.645
tf_mobilenetv3_small_075,5.292332,72.816
dla46_c,5.090697,72.607


In [9]:
df_results.to_csv("results_fp32_224.csv")

## plot

In [10]:
import seaborn as sns
sns.lmplot(y='top1', x='fp32',  
           data=df_results, logx=True,
           fit_reg=False, scatter_kws={'alpha':0.8})

# For various image size

In [11]:
# https://github.com/ryujaehun/pytorch-gpu-benchmark/blob/master/benchmark_models.py
def inference_imsize(modelname, benchmark, imsize):
    with torch.no_grad():
        model = timm.create_model(modelname,)
        model=model.to('cuda')
        model.eval()
        precision = "float"
        durations = []
        rand_loader = DataLoader(dataset=RandomDataset(BATCH_SIZE*(WARM_UP + NUM_TEST), imsize),
                         batch_size=BATCH_SIZE, shuffle=False,num_workers=8)
        print(f'Benchmarking Inference {modelname} ')
        for step,img in enumerate(rand_loader):
            img=getattr(img,precision)()
            torch.cuda.synchronize()
            start = time.time()
            model(img.to('cuda'))
            torch.cuda.synchronize()
            end = time.time()
            if step >= WARM_UP:
                durations.append((end - start)*1000)
        print(f'{modelname} model average inference time : {sum(durations)/len(durations)}ms')
        
        benchmark[modelname] = {"fp32": np.mean(durations), "top1": float(df_models[df_models["model"]==modelname]["top1"]), "imsize": imsize}
    return benchmark

In [12]:
modellist = df_models["model"]
benchmark = {}

# inference float precision
for i,modelname in tqdm(enumerate((modellist))):
    imsize = int(df_models[df_models["model"]==modelname]["img_size"])
    try:
        benchmark = inference_imsize(modelname, benchmark, imsize)
    except:
        print("pass {}".format(modelname))
benchmark

Benchmarking Inference beit_large_patch16_512 
beit_large_patch16_512 model average inference time : 187.82024145126343ms
Benchmarking Inference beit_large_patch16_384 
beit_large_patch16_384 model average inference time : 78.99953365325928ms
Benchmarking Inference tf_efficientnet_l2_ns 
tf_efficientnet_l2_ns model average inference time : 605.894718170166ms
Benchmarking Inference tf_efficientnet_l2_ns_475 
tf_efficientnet_l2_ns_475 model average inference time : 255.02083778381348ms
Benchmarking Inference beit_base_patch16_384 
beit_base_patch16_384 model average inference time : 30.691065788269043ms
Benchmarking Inference vit_large_patch16_384 
vit_large_patch16_384 model average inference time : 67.07225799560547ms
Benchmarking Inference cait_m48_448 
cait_m48_448 model average inference time : 482.82698154449463ms
Benchmarking Inference beit_large_patch16_224 
beit_large_patch16_224 model average inference time : 22.189147472381592ms
Benchmarking Inference tf_efficientnet_b7_ns 
tf

{'beit_large_patch16_512': {'fp32': 187.82024145126343,
  'top1': 90.695,
  'imsize': 512},
 'beit_large_patch16_384': {'fp32': 78.99953365325928,
  'top1': 90.601,
  'imsize': 384},
 'tf_efficientnet_l2_ns': {'fp32': 605.894718170166,
  'top1': 90.572,
  'imsize': 800},
 'tf_efficientnet_l2_ns_475': {'fp32': 255.02083778381348,
  'top1': 90.527,
  'imsize': 475},
 'beit_base_patch16_384': {'fp32': 30.691065788269043,
  'top1': 90.388,
  'imsize': 384},
 'vit_large_patch16_384': {'fp32': 67.07225799560547,
  'top1': 90.202,
  'imsize': 384},
 'cait_m48_448': {'fp32': 482.82698154449463, 'top1': 90.194, 'imsize': 448},
 'beit_large_patch16_224': {'fp32': 22.189147472381592,
  'top1': 90.157,
  'imsize': 224},
 'tf_efficientnet_b7_ns': {'fp32': 92.80160427093506,
  'top1': 90.134,
  'imsize': 600},
 'swin_large_patch4_window12_384': {'fp32': 54.970736503601074,
  'top1': 90.066,
  'imsize': 384},
 'swin_base_patch4_window12_384': {'fp32': 34.66608285903931,
  'top1': 90.063,
  'imsize': 

In [13]:
df_results = pd.DataFrame(benchmark).T
df_results
df_results.to_csv("results_fp32_imsizeall.csv")